<a href="https://colab.research.google.com/github/Nikhileshsandela/ds_project/blob/master/Markov_simulation1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%matplotlib notebook
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import scipy.sparse.linalg as sla

In [ ]:
import numpy as np
from discreteMarkovChain import markovChain

In [ ]:
P = np.array([[0.5,0.5],[0.6,0.4]])
mc = markovChain(P)
mc.computePi('linear')
print(mc.pi)

In [ ]:
class randomWalk(markovChain):

    def __init__(self,m,M):
        super(randomWalk, self).__init__()
        self.initialState = m
        self.m = m
        self.M = M
        self.uprate = 1.0
        self.downrate = 1.0

    def transition(self,state):
        rates = {}
        if self.m < state < self.M:
            rates[state+1] = self.uprate
            rates[state-1] = self.downrate
        elif state == self.m:
            rates[state+1] = self.uprate
        elif state == self.M:
            rates[state-1] = self.downrate
        return rates

In [ ]:
mc = randomWalk(0,5)
mc.computePi()
mc.printPi()

In [ ]:
from scipy.linalg import solve

In [ ]:
def GaussSeidel(A,B,x,n):
  L= np.tril(A)
  U=A-L
  for i in range(n):
    x = np.dot(np.linalg.inv(L),B-np.dot(U,x))
    print(x)
  return x

In [ ]:
A = eval(input('Enter the matrix A: '))
B = eval(input('Enter the matrix B: '))
x = eval(input('Enter the guess of x: '))
n = eval(input('Enter the number of iterations: '))
x = GaussSeidel(A,B,x,n)
print('Solution using the solve syntax: \n', solve(A,B))

In [ ]:
plt.rcParams['figure.figsize'] = [10, 6]
np.random.seed(42)

In [ ]:
transition_matrix = np.array([[0.2, 0.7, 0.1, 0.0],
                              [0.9, 0.0, 0.1, 0.0],
                              [0.2, 0.8, 0.0, 0.0],
                              [0.4, 0.5, 0.3, 0.0]])

In [ ]:
n_steps = 10**5
states = [0]
for i in range(n_steps):
    states.append(np.random.choice((0, 1, 2, 3), p=transition_matrix[states[-1]]))
states = np.array(states)

In [ ]:
def despine(ax, spines=('top', 'left', 'right')):
    for spine in spines:
        ax.spines[spine].set_visible(False)

fig, ax = plt.subplots()
width = 1000
offsets = range(1, n_steps, 5)
for i, label in enumerate(state_space):
    ax.plot(offsets, [np.sum(states[:offset] == i) / offset
            for offset in offsets], label=label)
ax.set_xlabel("number of steps")
ax.set_ylabel("Probability")
ax.legend(frameon=False)
despine(ax, ('top', 'right'))
plt.show()

# Left Eigen Vector Method

In [ ]:
state = {
    1 : "Up",
    2 : "Failed",
    3 : "Degraded",
    4 : "Critical"
}
state

{1: 'Up', 2: 'Failed', 3: 'Degraded', 4: 'Critical'}

In [ ]:
lambda1 = 0.0001
mu1 = 0.001

In [ ]:
M = np.array([-3*lambda1, mu1, 0, 0, mu1, -(mu1+2*lambda1), 2*lambda1, 0, 0, 2*mu1, -(2*mu1+lambda1), lambda1, 0, 0, 3*mu1, -3*mu1]).reshape(4,4)
M

array([[-0.0003,  0.001 ,  0.    ,  0.    ],
       [ 0.001 , -0.0012,  0.0002,  0.    ],
       [ 0.    ,  0.002 , -0.0021,  0.0001],
       [ 0.    ,  0.    ,  0.003 , -0.003 ]])

In [ ]:
eval, evec = sla.eigs(M.T, k=1, which='LM')

In [ ]:
eval

array([-0.00330442+0.j])

In [ ]:
evec

array([[ 0.19610495+0.j],
       [-0.70684465+0.j],
       [ 0.6456966 +0.j],
       [-0.21210703+0.j]])

In [ ]:
np.dot(evec.T, M).T

array([[-0.00064801+0.j],
       [ 0.00233571+0.j],
       [-0.00213365+0.j],
       [ 0.00070089+0.j]])

In [ ]:
u = (evec/evec.sum()).real
u

array([[-2.54186168],
       [ 9.16193753],
       [-8.36935228],
       [ 2.74927643]])

In [ ]:
ar = np.dot(u.T, M).T
ar

array([[  6.46106079],
       [-23.2883779 ],
       [ 21.27373582],
       [ -6.98828039]])

In [ ]:
M[M < 0] = 0
M

array([[0.    , 0.001 , 0.    , 0.    ],
       [0.001 , 0.    , 0.0002, 0.    ],
       [0.    , 0.002 , 0.    , 0.0001],
       [0.    , 0.    , 0.003 , 0.    ]])

In [ ]:
fig = plt.figure()
ax = fig.add_axes([0,0,1,1])
states = ['Up', 'Failed', 'Degraded', 'Critical']
probabilities = pi/N
ax.bar(states,probabilities, width = 0.30)
ax.set_xlabel("State")
ax.set_ylabel("Probability")
plt.legend(loc="upper right")
plt.show()

In [ ]:
# M /= M.sum()
M = np.asarray(M).astype('float64')
[M = x / np.sum(M) for x in M]

In [ ]:
N = 10**5
init = 0
present = init
pi = np.array([0, 0, 0, 0])
pi[init] = 1

i = 0
for i in range(N):
    # sum(M[present])
    # M[-1] = 1 - np.sum(M[0:-1])
    s = sum(M)
    [M = x / np.sum(M) for x in M]
    present = np.random.choice([0,1,2,3], p= M[present])
    pi[present]+=1
    past = present
#     if M.sum()!= 1.0:
#         M = M*(1./M.sum())
# return M


π = print("π= ", pi/N)

In [ ]:
#@title
N = 10**5
init = 0
present = init
pi = np.array([0, 0, 0, 0])
pi[init] = 1

i = 0
for i in range(N):
    present = np.random.choice([1,2,3,4], p=M[present], replace = False)
    pi[present]+=1
    past = present

π = print("π= ", pi/N)

In [ ]:
choices = [0,1,2,3]
weights = np.array([0,1,2,3])
normalized_weights = weights / np.sum(weights)

number_of_choices = 20
resample_counts = np.random.multinomial(number_of_choices,
                                        normalized_weights)

chosen = []
resample_index = 0
for resample_count in resample_counts:
    for _ in range(resample_count):
        chosen.append(choices[resample_index])
    resample_index += 1